# packages

In [1]:
from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries
from sunpy.timeseries.sources.goes import XRSTimeSeries
import astropy.units as u
from sunkit_instruments.goes_xrs import calculate_temperature_em
import matplotlib.pyplot as plt
import numpy as np
from sunpy.data import manager
import netCDF4 as nc
import os
import pandas as pd
from datetime import datetime, timedelta
import copy
from matplotlib import colormaps
list(colormaps)
#from colorspacious import cspace_converter
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from matplotlib.ticker import LogFormatter
from matplotlib.ticker import LogFormatterMathtext
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import seaborn as sns
import re
import glob
import logging


/home/pjgonzalesp/miniforge3/envs/sunpy/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Funciones

In [2]:
##############################
##    1. Download GOES data   #
##############################

## 1.0. Ensure 1 min resolution (C1)
def ensure_1min_resolution(ts):
    """
    Revisa si el TimeSeries está en resolución de 1 minuto.
    Si no, lo re-muestrea a 1 min con la media.

    Check if the TimeSeries has a resolution of 1 minute.
    If no, it will be shown again in 1 minute with the media
    """
    # Pasar a DataFrame
    df = ts.to_dataframe()
    
    # Calcular la resolución actual (diferencia entre los 2 primeros tiempos)
    current_res = (df.index[1] - df.index[0]).total_seconds()
    
    if abs(current_res - 60) < 1:  # ya es 1 min (tolerancia de 1s)
        print("Resolution = 1 minute")
        return ts
    else:
        print(f"Resolution detected: {current_res:.2f} s → resampling at 1 min")
        df_resampled = df.resample("1min").mean()
        return TimeSeries(df_resampled, ts.meta)

## 1.1. Download data (C1)
def Download_Data(start_time, end_time, resolution="avg1m", log_file=None, output_dir=None):
    
    """
    Descarga datos GOES entre start_time y end_time, asegura resolución 1min,
    guarda gráficas en block_dir/data_graphs y log de errores en block_dir.

    Entrada / Input:
        start_time (str): Tiempo inicial del intervalo de búsqueda, en formato compatible con SunPy.
        end_time (str): Tiempo final del intervalo de búsqueda, en formato compatible con SunPy.
        resolution (str, opcional / optional): Resolución temporal de los datos GOES. 
            Opciones válidas: "flx1s", "avg1m". Por defecto es 'avg1m'.

    Salida / Output:
        TimeSeries: Objeto de SunPy que contiene los datos XRS del satélite GOES 
                    dentro del intervalo de tiempo especificado.
                    con la resolución especificada.

    Descripción / Description:
        Esta función busca, descarga y carga datos del instrumento GOES (X-Ray Sensor, XRS)
        en el intervalo de tiempo especificado y con la resolución deseada. Utiliza Fido para la búsqueda
        y retorna un objeto TimeSeries con los datos.

        This function searches for, downloads, and loads data from the GOES (X-Ray Sensor, XRS)
        within the specified time interval and chosen resolution. It uses Fido for querying and 
        returns a TimeSeries object with the data.
    """
    try:
        # ===== Directorios =====
        if output_dir is not None:
            os.makedirs(output_dir, exist_ok=True)
            graph_dir = os.path.join(output_dir, "data_graphs")
            os.makedirs(graph_dir, exist_ok=True)
            log_file = os.path.join(output_dir, "errores_goes.log")
        else:
            graph_dir = "data_graphs"
            os.makedirs(graph_dir, exist_ok=True)
            log_file = "errores_goes.log"

        # ===== Validar resolución =====
        valid_resolutions = ["flx1s", "avg1m"]
        if resolution not in valid_resolutions:
            raise ValueError(f"Resolución no válida. Usa una de: {valid_resolutions}")

        # ===== Buscar y descargar datos =====
        print(f"Buscando datos de: {start_time}")
        result = Fido.search(a.Time(start_time, end_time), a.Instrument.goes, a.Resolution(resolution))

        if len(result[0]) == 0:
            msg = f"No hay datos GOES para {start_time} - {end_time}. Día saltado."
            print(msg)
            with open(log_file, "a") as f:
                f.write(msg + "\n")
            return None, None  

        print(f"Descargando datos de {start_time}...")
        files = Fido.fetch(result)

        if len(files) == 0:
            msg = f"No se descargaron archivos GOES para {start_time}. Día saltado."
            print(msg)
            with open(log_file, "a") as f:
                f.write(msg + "\n")
            return None, None

        # ===== Cargar datos en TimeSeries =====
        try:
            ts = TimeSeries(files[0], source="XRS")
        except Exception as e:
            msg = f"Error al abrir archivo GOES de {start_time}: {e}"
            print(msg)
            with open(log_file, "a") as f:
                f.write(msg + "\n")
            return None, None

        # ===== Asegurar resolución 1 min =====
        goes_ts = ensure_1min_resolution(ts)

        # ===== Guardar gráfica =====
        fig, ax = plt.subplots(figsize=(12,6))
        goes_ts.plot(axes=ax)
        safe_time = start_time.replace(':','-').replace(' ','_')
        output_file = os.path.join(graph_dir, f"GOES_{safe_time}.png")
        fig.savefig(output_file, dpi=150, bbox_inches="tight")
        plt.close(fig)
        print(f"Gráfica guardada en {output_file}")

        # ===== Extraer observatorio =====
        try:
            meta0 = goes_ts.meta.metas[0]
            platform = meta0.get("platform", "g16")
            numero = int("".join(filter(str.isdigit, platform)))
            observatory = f"GOES-{numero}"
        except Exception:
            observatory = None
        print(f"Observatorio encontrado: {observatory}")

        return goes_ts, observatory

    except Exception as e:
        msg = f"Error inesperado al descargar datos GOES de {start_time}: {e}"
        print(msg)
        if block_dir is not None:
            with open(log_file, "a") as f:
                f.write(msg + "\n")
        return None, None  #  siempre devuelve una tupla
    

## 1.2. Truncar datos
def Truncate_Data(goes_ts, flare_start_time, flare_end_time):
    
    """
    Entrada / Input:
        goes_ts (TimeSeries): Objeto TimeSeries con los datos GOES completos.
        flare_start_time (str): Tiempo de inicio de la fulguración (en formato compatible con SunPy).
        flare_end_time (str): Tiempo de fin de la fulguración (en formato compatible con SunPy).

    Salida / Output:
        TimeSeries: Objeto TimeSeries con los datos recortados al intervalo de la fulguración.

    Descripción / Description:
        Esta función recorta un conjunto de datos GOES a un intervalo de tiempo específico
        correspondiente al inicio y fin de una fulguración solar. Si el intervalo no contiene datos,
        se lanza una excepción.

        This function trims a GOES TimeSeries dataset to a specific time interval
        corresponding to the start and end of a solar flare. If the interval contains no data,
        an exception is raised.
    """

    # Seleccionar el rango de interés / Select the time range of interest
    goes_flare = goes_ts.truncate(flare_start_time, flare_end_time)

    # Verificar si hay datos disponibles  / Check if data is available
    if len(goes_flare.to_dataframe()) == 0:
        raise ValueError("El rango de datos seleccionado está vacío. Revisa las fechas.")
        # The selected time range is empty. Please check the input times.

    # Visualizar los datos truncados / Plot the trimmed data
    goes_flare.peek()

    return goes_flare

## 2. Background (1)
def running_difference(goes_ts, Dif_time=5, plot=False, block_dir=None, start_time=None):
    """
    Calcula las diferencias de flujo de rayos X GOES a un intervalo definido (default 5 min).
    
    Parámetros
    ----------
    goes_ts : XRSTimeSeries
        Serie temporal original de GOES.
    Dif_time : int, optional
        Intervalo de diferencia en número de pasos para restar flux (default=5).
    plot : bool, optional
        Si True, guarda gráficas comparativas original vs corregido.
    block_dir : str, optional
        Carpeta base donde guardar gráficas.
    start_time : str, optional
        Tiempo inicial usado para nombrar archivos.
        
    Retorna
    -------
    goes_diff_ts : XRSTimeSeries
        Serie temporal corregida con las diferencias.
    """

    import matplotlib.dates as mdates
    from matplotlib.ticker import LogFormatterMathtext

    # --- 1. Extraer datos ---
    df = goes_ts.to_dataframe()
    flux_xrsa = df["xrsa"]
    flux_xrsb = df["xrsb"]
    npts = len(df)

    # --- 2. Calcular diferencias ---
    diffa = np.array(flux_xrsa[Dif_time:]) - np.array(flux_xrsa[:npts - Dif_time])
    diffb = np.array(flux_xrsb[Dif_time:]) - np.array(flux_xrsb[:npts - Dif_time])

    # --- 3. Llenar arreglos completos ---
    diffa_full = np.zeros(npts)
    diffb_full = np.zeros(npts)
    diffa_full[Dif_time:] = diffa
    diffb_full[Dif_time:] = diffb

    # --- 4. Crear DataFrame corregido ---
    df_diff = pd.DataFrame({'xrsa': diffa_full, 'xrsb': diffb_full}, index=df.index)

    # --- 5. Crear TimeSeries corregida ---
    units = {'xrsa': u.W / u.m**2, 'xrsb': u.W / u.m**2}
    goes_diff_ts = XRSTimeSeries(df_diff, units=units, meta=goes_ts.meta)

    # --- 6. Función auxiliar para graficar ---
    def save_plot(df_orig, df_corr, output_file, title="", logscale=False, positive_only=False):
        df_o, df_c = df_orig.copy(), df_corr.copy()
        if positive_only:
            df_o = df_o.clip(lower=1e-9)
            df_c = df_c.clip(lower=1e-9)

        fig, ax = plt.subplots(figsize=(12,6))
        ax.plot(df_o.index, df_o['xrsa'], label='XRSA (original)', color='blue')
        ax.plot(df_c.index, df_c['xrsa'], label='XRSA (corrected)', color='blue', linestyle='--')
        ax.plot(df_o.index, df_o['xrsb'], label='XRSB (original)', color='red')
        ax.plot(df_c.index, df_c['xrsb'], label='XRSB (corrected)', color='red', linestyle='--')

        date_only = df_orig.index[0].strftime("%Y-%m-%d")
        ax.set_xlabel(f"Time (UTC) — {date_only}")
        ax.set_ylabel("Flux [W/m²]")
        ax.set_title(title)
        ax.legend()
        ax.grid(True, which='both', ls='--', alpha=0.6)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

        if logscale:
            ax.set_yscale('log', base=10)
            ax.yaxis.set_major_formatter(LogFormatterMathtext())

        plt.tight_layout()
        fig.savefig(output_file, dpi=150, bbox_inches="tight")
        plt.close(fig)
        print(f"Plot saved at: {output_file}")

    # --- 7. Guardar gráficas si plot=True ---
    if plot and block_dir is not None and start_time is not None:
        graph_dir = os.path.join(block_dir, "data_graphs")
        os.makedirs(graph_dir, exist_ok=True)

        safe_time = start_time.replace(':','-').replace(' ','_')
        df_corr = df_diff

        # Linear
        save_plot(df, df_corr, os.path.join(graph_dir, f"GOES_diff_linear_{safe_time}.png"),
                  title=f"GOES Data Comparison (Δt={Dif_time} steps)")

        # Logarithmic
        save_plot(df, df_corr, os.path.join(graph_dir, f"GOES_diff_log_{safe_time}.png"),
                  title=f"GOES Data Comparison (Δt={Dif_time} steps) [Log Y]",
                  logscale=True)

        # Positive only
        save_plot(df, df_corr, os.path.join(graph_dir, f"GOES_diff_positive_{safe_time}.png"),
                  title=f"GOES Data Comparison [Positive only, Log Y]",
                  logscale=True,
                  positive_only=True)

    return goes_diff_ts


#####################################
# 5. Temperatura y Emission Measure #
#####################################

## 5.1. Calcular la Temperatura y Medida de emisión con los datos corregidos (C1)
#Calculate_Tem_EM(goes_flare_corrected, abundance='photospheric')
#Calculate_Tem_EM(goes_flare_corrected, abundance='coronal')
def Calculate_Tem_EM(goes_flare_corrected, abundance='coronal'):
    
    """
    Entrada / Input:
        goes_flare_corrected: Objeto TimeSeries corregido que contiene los datos del flare solar en los
        canales XRSA y XRSB. Debe ser compatible con la función `calculate_temperature_em` de SunPy.
        abundance (str): Tipo de abundancia elemental a usar en el cálculo (por defecto: 'coronal').
                         Otras opciones posibles incluyen 'photospheric', dependiendo del modelo de SunPy.

    Salida / Output:
        temp_em: Objeto que contiene la temperatura (T) y medida de emisión (EM) derivadas a partir de los
        datos GOES corregidos.

    Descripción / Description:
        Esta función utiliza los datos corregidos del satélite GOES para calcular la temperatura del plasma
        y la medida de emisión (EM) durante un evento de fulguración solar. Permite especificar el modelo
        de abundancia elemental a utilizar en el cálculo.

        This function uses corrected GOES data to compute the plasma temperature and emission measure (EM)
        during a solar flare. It allows specifying the elemental abundance model to be used in the calculation.

    Notas / Notes:
        - Usa la función `calculate_temperature_em` de SunPy.
        - El parámetro `abundance` controla el modelo de abundancias (por ejemplo, 'coronal' o 'photospheric').
        - Se desactiva temporalmente la verificación del hash de calibración del instrumento.
        - Los datos deben estar previamente corregidos y limpios.
    """

    print(f'Ahora vamos a calcular la T y EM con el modelo de abundancias:{abundance}')
    #  Saltar la verificación del hash temporalmente
    with manager.skip_hash_check():
        #temp_em = calculate_temperature_em(goes_flare_corrected, abundance='coronal')
        temp_em = calculate_temperature_em(goes_flare_corrected, abundance)
    
    print(f'se calculó T y EM con el modelo de abundancias:{abundance}')
    #print(temp_em)
    return temp_em


# mostrar un rango de datos de un TimeSeries
def show_range_ts(ts, start_idx, end_idx, height=300):
    df = ts.to_dataframe()
    subdf = df.iloc[start_idx:end_idx]
    display(HTML(f"""
    <div style="height:{height}px; overflow:auto; border:1px solid #ccc; padding:10px">
        {subdf.to_html()}
    </div>
    """))

# Uso:
#show_range_ts(goes_ts01, 0, 10)

#####################################
#     6. Calcula tiempos de FAI     #
#####################################

def calcular_fai_times(df, 
                       T_min=7, T_max=14, 
                       EM_threshold=0.005, 
                       col_T="T_cor", 
                       col_EM="EM_cor_norm"):
    """
    Calcula los tiempos en los que se cumplen los criterios del índice FAI.
    """

    # Copiar para no modificar el original
    df_fai = df.copy()

    # Condiciones del criterio FAI
    fai_condition = (
        (df_fai[col_T] >= T_min) & 
        (df_fai[col_T] <= T_max) & 
        (df_fai[col_EM] > EM_threshold)
    )

    # Selección
    df_fai_selected = df_fai[fai_condition]
    fai_times = df_fai_selected.index

    # Mostrar los resultados como en tu código original
    print(f"Se encontraron {len(df_fai_selected)} puntos que cumplen el criterio FAI.\n")
    print(df_fai_selected[[col_T, col_EM]].head())

    return fai_times, df_fai_selected

########################
# 7. Descargar Flares #
#######################

# Configurar logging básico
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

## 7.1. Descarga de datos de flares
def get_flares(start_time, end_time, output_dir=None):
    """
    Busca fulguraciones solares reportadas por GOES en el intervalo dado.
    y guarda un log si no se encuentran flares o si hay errores.
    
    Parameters
    ----------
    start_time : str
        Tiempo inicial (YYYY-MM-DD o compatible con SunPy).
    end_time : str
        Tiempo final (YYYY-MM-DD o compatible con SunPy).
    log_file : str o None, opcional
        Ruta del archivo .log donde guardar mensajes si no hay flares o hay errores.
        Si es None, no se escribe log en disco (solo en consola).

    
    Returns
    -------
    pd.DataFrame or None
        DataFrame con columnas ['StartTime', 'EndTime', 'Class', 'Observatory', 'PeakTime'] 
        o None si no se encontraron flares.
    """
    try:
        logging.info(f"Searching for GOES flares between {start_time} and {end_time}...")
        result = Fido.search(a.Time(start_time, end_time), a.hek.FL, a.hek.OBS.Observatory == "GOES")

        if not result or len(result) == 0 or len(result[0]) == 0:
            msg = f"No solar flares found between {start_time} and {end_time}."
            logging.info(msg)
            if output_dir is not None:
                os.makedirs(output_dir, exist_ok=True)
                log_path = os.path.join(output_dir, "no_flares.log")
                with open(log_path, "a") as f:
                    f.write(msg + "\n")
            return None

        # Filtrar columnas 1D
        names = [name for name in result[0].colnames if len(result[0][name].shape) <= 1]
        table = result[0][names].to_pandas()

        # Seleccionar columnas de interés
        flare_data = table[[
            "event_starttime",
            "event_endtime",
            "fl_goescls",
            "obs_observatory",
            "event_peaktime"
        ]]

        # Renombrar columnas
        flare_data.columns = ["StartTime", "EndTime", "Class", "Observatory", "PeakTime"]

        logging.info(f"Found {len(flare_data)} GOES solar flares between {start_time} and {end_time}.")
        return flare_data

    except Exception as e:
        msg = f"❌ Error retrieving flares between {start_time} and {end_time}: {e}"
        logging.error(msg)

        if output_dir is not None:
            os.makedirs(output_dir, exist_ok=True)
            log_path = os.path.join(output_dir, "no_flares.log")
            with open(log_path, "a") as f:
                f.write(msg + "\n")
        return None
    

def build_full_dataframe(goes_ts, goes_corrected, temp_em_cor, temp_em_phot,
                         clip_negative=True, normalize_em=False):
    """
    Combina datos originales, corregidos y parámetros de temperatura/EM
    en un solo DataFrame.

    Parámetros
    ----------
    goes_ts : sunpy.timeseries.TimeSeries
        Serie temporal GOES remuestreada (contiene 'xrsa', 'xrsb').
    goes_corrected : sunpy.timeseries.TimeSeries
        Serie temporal con GOES corregido (xrsa, xrsb corregidos).
    temp_em_cor : sunpy.timeseries.TimeSeries
        Serie temporal con temperatura y EM coronal.
    temp_em_phot : sunpy.timeseries.TimeSeries
        Serie temporal con temperatura y EM fotosférica.
    clip_negative : bool, opcional
        Si True, reemplaza valores negativos con NaN (en lugar de 0).
    normalize_em : bool, opcional
        Si True, normaliza EM a unidades de 1e49 cm^-3.

    Retorna
    -------
    pd.DataFrame
        DataFrame combinado con todas las columnas.
    """

    # Originales
    df_original = goes_ts.to_dataframe()[['xrsa', 'xrsb']]

    # Corregidos
    df_corr = goes_corrected.to_dataframe().rename(
        columns={'xrsa': 'xrsa_corr', 'xrsb': 'xrsb_corr'}
    )

    # Coronal
    df_cor = temp_em_cor.to_dataframe()[['temperature', 'emission_measure']].rename(
        columns={'temperature': 'T_cor', 'emission_measure': 'EM_cor'}
    )

    # Fotosférica
    df_phot = temp_em_phot.to_dataframe()[['temperature', 'emission_measure']].rename(
        columns={'temperature': 'T_phot', 'emission_measure': 'EM_phot'}
    )

    # Combinar todo
    df_full = pd.concat([df_original, df_corr, df_cor, df_phot], axis=1)

    # Opcional: reemplazar valores negativos
    if clip_negative:
        df_full = df_full.mask(df_full < 0, np.nan)   # ahora quedan NaN y no 0

    # Opcional: normalizar EM
    if normalize_em:
        df_full['EM_cor_norm'] = df_full['EM_cor'] / 1e49
        df_full['EM_phot_norm'] = df_full['EM_phot'] / 1e49

    return df_full


# agrupa por clases de flares en 2 grupos por clase
def assign_flare_group(flare_class):
    """
    Asigna un grupo a un flare según su clase y subnivel.
    
    Ejemplos:
    - C3.0 → C1-4
    - C7.5 → C5-9
    - X2.0 → X1-4
    - X7.0 → X5+
    """
    group_ranges = {
        "A": [(1, 4), (5, 9)],
        "B": [(1, 4), (5, 9)],
        "C": [(1, 4), (5, 9)],
        "M": [(1, 4), (5, 9)],
        "X": [(1, 4), (5, 1000)]  # 1000 actúa como "infinito"
    }

    try:
        letter = flare_class[0]  # A, B, C, M, X
        number = float(flare_class[1:])  # acepta decimales, ej: "2.7" → 2.7
    except:
        return flare_class  # si algo raro viene en el CSV

    if letter not in group_ranges:
        return flare_class

    for (low, high) in group_ranges[letter]:
        if low <= number <= high:
            if high >= 1000:  # caso abierto (ej: X5+)
                return f"{letter}{low}+"
            return f"{letter}{low}-{high}"
    
    return flare_class


#######################
### cuenta los días ###
#######################

def count_days(start_date, end_date):
    """
    Counts the number of days between two dates.

    Parameters:
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
    - int: Number of days between start_date and end_date
    """
    # Convert the strings to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Calculate the difference
    difference = end - start
    
    return difference.days

# Example usage
#days = count_days("1980-01-05", "2025-08-20")
#print(f"Days between the dates: {days}")

# Dataframe con todas las fechas
def all_dates_dataframe(start_date, end_date):
    """
    Crea un DataFrame con todas las fechas entre start_date y end_date (inclusive).

    Parámetros:
    - start_date (str): Fecha inicial en formato 'YYYY-MM-DD'
    - end_date (str): Fecha final en formato 'YYYY-MM-DD'

    Retorna:
    - pd.DataFrame con columnas ['start_time', 'end_time']
    """
    # Convertir a objetos datetime
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Generar todas las fechas día a día
    all_days = [start + timedelta(days=i) for i in range((end - start).days + 1)]
    
    # Crear DataFrame
    data = {
        "start_time": [d.strftime("%Y-%m-%d 00:00:00") for d in all_days],
        "end_time":   [d.strftime("%Y-%m-%d 23:59:00") for d in all_days]
    }
    return pd.DataFrame(data)

#############################################
# elige los días cada cierto paso según     #
# el número de días que quiera analizar (n) #
#############################################

def select_dates(start_date, end_date, n=10):
    """
    Selects n dates evenly spaced between start_date and end_date.

    Parameters:
    - start_date (str): Start date in 'YYYY-MM-DD' format
    - end_date (str): End date in 'YYYY-MM-DD' format
    - n (int): Number of dates to select (default 10)

    Returns:
    - list of str: List of dates in 'YYYY-MM-DD' format
    """
    # Convert strings to datetime objects
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Calculate total number of days
    total_days = (end - start).days
    
    # Calculate integer step
    #step = total_days // (n - 1)  # n-1 intervals for n dates
    
    if n < 2:
        return [start_date]

    # step aproximado al entero más cercano
    step = max(1, int(round(total_days / (n - 1))))
    print(step)
    # Generate the dates
    dates = [start + timedelta(days=i*step) for i in range(n)]
    print(len(dates))
    # Convert to strings
    return [d.strftime("%Y-%m-%d") for d in dates]


# guarda los datos seleccionados segun el paso en un df
def dates_to_dataframe(dates_list):
    """
    Converts a list of dates into a DataFrame with start_time and end_time.

    Parameters:
    - dates_list (list of str): List of dates in 'YYYY-MM-DD' format

    Returns:
    - pd.DataFrame: DataFrame with columns ['start_time', 'end_time']
    """
    data = {
        "start_time": [f"{date} 00:00:00" for date in dates_list],
        "end_time":   [f"{date} 23:59:00" for date in dates_list]
    }
    return pd.DataFrame(data)


# Divide un dateframen en lotes:
def chunk_dataframe(df, chunk_size=20):
    """
    Divide un DataFrame en bloques (chunks) de tamaño chunk_size.
    Devuelve una lista de DataFrames.
    """
    return [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# unir todos los bloques al final
def combine_blocks(output_dir, pattern, final_name, time_column=None):
    """
    Une todos los CSV de bloques en un único archivo final, manteniendo resolución de 1 min.
    Si los CSV tienen la primera columna como fecha sin nombre, se renombra como 'date'.
    """
    import os, glob, pandas as pd

    # Buscar todos los CSV que coincidan con el patrón (también dentro de subcarpetas)
    files = sorted(glob.glob(os.path.join(output_dir, "**", f"*{pattern}*.csv"), recursive=True))
    output_file = os.path.join(output_dir, final_name)

    # Evitar incluir el archivo combinado si ya existe
    files = [f for f in files if os.path.abspath(f) != os.path.abspath(output_file)]

    if not files:
        print(f"No se encontraron archivos con patrón {pattern}")
        return

    df_list = []
    for f in files:
        df = pd.read_csv(f)
        # Si la primera columna no tiene nombre o se llama 'Unnamed: 0', la renombramos como 'date'
        first_col = df.columns[0]
        if first_col.startswith("Unnamed") or first_col == "":
            df.rename(columns={first_col: "date"}, inplace=True)
        df_list.append(df)

    combined = pd.concat(df_list, ignore_index=True)

    # Si no se especifica columna de tiempo, usamos la primera (ya llamada 'date')
    if time_column is None:
        time_column = combined.columns[0]

    # Convertir a datetime y ordenar
    combined[time_column] = pd.to_datetime(combined[time_column], errors="coerce")
    combined = combined.sort_values(by=time_column).reset_index(drop=True)

    # 🔹 Eliminar duplicados exactos (mismo timestamp)
    combined = combined.drop_duplicates(subset=[time_column])

    # 🔹 Verificar resolución de 1 min
    diffs = combined[time_column].diff().dropna()
    if (diffs.dt.total_seconds() == 60).all():
        print("⏱️ Resolución consistente de 1 minuto.")
    else:
        print("⚠️ Hay diferencias en la resolución temporal (no todos los pasos son 1 min).")

    # Guardar resultado final
    combined.to_csv(output_file, index=False)
    print(f"✅ {final_name} creado con {len(combined)} filas (desde {len(files)} archivos).")
    return combined

# Example usage
#dates_list = select_dates("1980-01-05", "2025-08-20", n=10)
#df_intervals = dates_to_dataframe(dates_list)

#print(df_intervals)

# Función completa

In [3]:
def download_goes_flare_data(start_time, end_time, 
                      resolution="avg1m",  #resolución de descarga de los datos GOES (1min)
                      Dif_time=5,            #Difference time Δt (for background)
                      plot_diff=True,
                      output_dir=None):
    """
    Pipeline completo para análisis de datos GOES y cálculo de FAI y anticipación.

    Parámetros:
    -----------
    start_time : str
        Tiempo inicial (ej: "2017-09-06 12:00:00")
    end_time : str
        Tiempo final (ej: "2017-09-06 12:15:00")
    resolution : str, opcional
        Resolución de datos GOES, default "avg1m". Opciones: ["flx1s", "avg1m"]
    Dif_time : int, opcional
        Ventana en pasos para calcular diferencias (default 5).
    plot_diff : bool, opcional
        Si True, grafica las diferencias calculadas.
    
    Retorna:
    --------
    dict con:
        - df_full : DataFrame con todos los cálculos
        - df_flare_data : DataFrame de flares GOES en el intervalo
    """

    # 1. Descargar datos GOES
    print("1. Descargar datos GOES")
    data = Download_Data(start_time, end_time, resolution, log_file="errores_goes.log", output_dir=output_dir)
    # para días sin datos GOES
    if data is None:
        print(f"No hay datos GOES para {start_time} - {end_time}. Día saltado.\n")
        return None
    else:
        print(f"Se encontraron datos  GOES para {start_time} - {end_time}. Día saltado.\n")
        goes_ts, observatory = data
        # Convertir TimeSeries a DataFrame para inspección
        df_goes = goes_ts.to_dataframe()

        # Número de registros
        print(f"Número de registros: {len(df_goes)}")

        # Columnas disponibles
        print(f"Columnas disponibles: {list(df_goes.columns)}")
        
    print("2. Restar Background")
    # 2. Calcular diferencias
    goes_ts_corrected_diff = running_difference(goes_ts, Dif_time=Dif_time, plot=plot_diff, block_dir=output_dir, start_time=start_time)
                            

    # 3. Calcular T y EM (coronal y fotosférico)
    print("3. USAR FUNCIÓN SUNPY calculate_t_em")
    temp_em_cor = Calculate_Tem_EM(goes_ts_corrected_diff, abundance='coronal')
    temp_em_phot = Calculate_Tem_EM(goes_ts_corrected_diff, abundance='photospheric')

    print("4. CONSTRUIR df_full")
    # 4. Construir dataframe completo
    df_full = build_full_dataframe(goes_ts, goes_ts_corrected_diff, temp_em_cor, temp_em_phot,
                         clip_negative=True, normalize_em=True)

    print(f"5. añadiendo observatorio: GOES: {observatory}")
    # 5. Añadir columna del observatorio
    df_full["observatory"] = observatory
    # Reemplazar NaN por "Unknown"
    df_full["observatory"] = df_full["observatory"].fillna("Unknown")
    # Mover "observatory" al inicio
    cols = ["observatory"] + [col for col in df_full.columns if col != "observatory"]
    df_full = df_full[cols]
    #print(df_full)
    print(f"se añadió observatorio: GOES: {observatory}")

    #print(f"6. Normalizando EM")
    # 6. Normalizar EM
    #df_full['EM_cor_norm'] = df_full['EM_cor'] / 1e49
    #df_full['EM_phot_norm'] = df_full['EM_phot'] / 1e49
    
    print(f"6. Descargando flares: {start_time} - {end_time}")
    # 6. Descargar flares
    flare_data = get_flares(start_time, end_time, output_dir=output_dir)
    
    #verifica si encontró flares para el día
    if flare_data is None:
        print(f"No se encontraron flares para el intervalo {start_time} - {end_time}. Saltando...")
        # Crear DataFrames vacíos para todos los resultados que dependen de flares
        df_flare_data = pd.DataFrame()
    
        
        # Retornar diccionario con flares vacíos
        return {
            "df_full": df_full,
            "df_flare_data": df_flare_data,
        }
    

    # Si flare_data no es None continúa
    print(f"Se descargaron flares para el intervalo {start_time} - {end_time}.")
    df_flare_data = flare_data[
        flare_data['Class'].notna() & 
        (flare_data['Class'].str.strip() != "") &
        (flare_data['Observatory'] == "GOES")
    ]
    print(f"7. Se filtraron solo flares de GOES.")
      

    # Retornar todo como dict
    return {
        "df_full": df_full,
        "df_flare_data": df_flare_data,
    }

# lista de fechas desde CSV

In [4]:
# Leer lista de fechas desde CSV
csv_path = "2025-10-24_Download_for_specific_days/Specific_days.csv"
df_fechas = pd.read_csv(csv_path)
# Asegurar que la columna se llame "date" y sea datetime
df_fechas["date"] = pd.to_datetime(df_fechas["date"])
df_days = pd.DataFrame({
    "start_time": df_fechas["date"],
    "end_time": df_fechas["date"] + pd.Timedelta(days=1)
})
print(df_days.head())
print(f"Total de fechas cargadas: {len(df_days)}")

  start_time   end_time
0 2000-06-06 2000-06-07
1 2000-06-07 2000-06-08
2 2000-06-18 2000-06-19
3 2000-07-11 2000-07-12
4 2000-07-12 2000-07-13
Total de fechas cargadas: 185


# NUeva carperta

In [5]:
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y-%m-%d")

# Nombre de la carpeta principal
output_dir = f"{fecha_actual}_Download_for_specific_days"

# Crear carpeta principal (si no existe) y mostrar mensaje
if os.path.exists(output_dir):
    print(f"⚠️ La carpeta principal ya existía: {output_dir}")
else:
    os.makedirs(output_dir, exist_ok=True)
    print(f"📁 Carpeta principal creada: {output_dir}")

# Crear subcarpeta dentro de la principal
subfolder = os.path.join(output_dir, "Data")

# Crear subcarpeta (si no existe) y mostrar mensaje
if os.path.exists(subfolder):
    print(f"⚠️ La subcarpeta ya existía: {subfolder}")
else:
    os.makedirs(subfolder, exist_ok=True)
    print(f"📂 Subcarpeta creada: {subfolder}")

⚠️ La carpeta principal ya existía: 2025-10-24_Download_for_specific_days
⚠️ La subcarpeta ya existía: 2025-10-24_Download_for_specific_days/Data


# correr función completa para dias en csv

In [6]:
# función para procesar y guardar datos por mes (días específicos)
def process_goes_by_month_specific_days(df_days, output_dir):
    """
    Procesa los datos GOES para días específicos agrupando por mes.

    Crea una carpeta por mes (YYYY_MM) dentro de output_dir,
    y guarda los CSV 'df_full' y 'df_flare_data' para cada mes.

    Parámetros:
    - df_days: DataFrame con columna ['date'] con los días a descargar
    - output_dir: ruta base donde se guardarán los resultados
    """
    import os
    import pandas as pd
    import sys

    # Asegurar que la columna de fecha es tipo datetime
    df_days["date"] = pd.to_datetime(df_days["date"])
    df_days["start_time"] = df_days["date"]
    df_days["end_time"]   = df_days["start_time"] + pd.Timedelta(days=1)

    # Crear columna auxiliar con el mes en formato YYYY_MM
    df_days["month"] = df_days["start_time"].dt.strftime("%Y_%m")

    total_days = len(df_days)
    print(f"\n📅 Total de días a descargar: {total_days}")

    day_counter = 0  # contador de progreso

    # Agrupar por mes
    for month, df_month in df_days.groupby("month"):
        print("\n" + "="*40)
        print(f"🟦 Procesando mes {month} ({len(df_month)} días)")
        print("="*40)

        # Carpeta para este mes
        month_dir = os.path.join(output_dir, f"Month_{month}")
        os.makedirs(month_dir, exist_ok=True)

        # Archivos esperados
        file_full_month  = os.path.join(month_dir, f"df_full_{month}.csv")
        file_flare_month = os.path.join(month_dir, f"df_flare_data_{month}.csv")

        # Leer archivos existentes si hay
        if os.path.exists(file_full_month):
            existing_full = pd.read_csv(file_full_month)

            # Asegurar que exista columna 'date'
            first_col = existing_full.columns[0]
            if first_col.startswith("Unnamed") or first_col == "":
                existing_full.rename(columns={first_col: "date"}, inplace=True)
            if 'date' not in existing_full.columns:
                existing_full.insert(0, 'date', pd.NaT)
            existing_full['date'] = pd.to_datetime(existing_full['date'], errors='coerce')
            downloaded_days = existing_full['date'].dt.date.dropna().unique()
        else:
            existing_full = pd.DataFrame(columns=['date'])
            downloaded_days = []

        if os.path.exists(file_flare_month):
            existing_flares = pd.read_csv(file_flare_month)
            if 'StartTime' not in existing_flares.columns:
                first_col = existing_flares.columns[0]
                if first_col.startswith("Unnamed") or first_col == "":
                    existing_flares.rename(columns={first_col: "StartTime"}, inplace=True)
                else:
                    existing_flares['StartTime'] = pd.to_datetime(existing_flares[first_col], errors='coerce')
            existing_flares['StartTime'] = pd.to_datetime(existing_flares['StartTime'], errors='coerce')
        else:
            existing_flares = pd.DataFrame(columns=['StartTime'])

        # Filtrar días que faltan
        df_month_to_download = df_month[~df_month['start_time'].dt.date.isin(downloaded_days)]

        # Guardar días ya existentes
        already_downloaded = df_month['start_time'].dt.date[df_month['start_time'].dt.date.isin(downloaded_days)].tolist()
        if already_downloaded:
            print(f"⚠️ Días ya descargados y saltados: {already_downloaded}")

        if df_month_to_download.empty:
            print(f"⚠️ Todos los días del mes {month} ya fueron descargados. Saltando...")
            continue

        list_df_full = []
        list_df_flare_data = []

        # Procesar cada día
        for idx, row in df_month_to_download.iterrows():
            start_time = row["start_time"].strftime("%Y-%m-%d %H:%M:%S")
            end_time   = row["end_time"].strftime("%Y-%m-%d %H:%M:%S")
            date_str   = row["start_time"].strftime("%Y-%m-%d")
            day_counter += 1
            print(f"Día {day_counter} de {total_days}: {date_str}", file=sys.stderr)

            try:
                results = download_goes_flare_data(
                    start_time, end_time,
                    resolution="avg1m",
                    Dif_time=5,
                    plot_diff=True,
                    output_dir=month_dir
                )
            except Exception as e:
                print(f"❌ Error en {date_str}: {e}", file=sys.stderr)
                continue

            if results is None:
                print(f"⚠️ Sin resultados para {date_str}", file=sys.stderr)
                continue

            df_full = results["df_full"]
            df_flare_data = results["df_flare_data"]

            # Asegurarse de que df_full tenga columna 'date'
            if 'date' not in df_full.columns:
                if isinstance(df_full.index, pd.DatetimeIndex):
                    df_full = df_full.reset_index().rename(columns={'index': 'date'})
                else:
                    df_full.insert(0, 'date', row["start_time"])

            if not df_full.empty:
                list_df_full.append(df_full)
            if not df_flare_data.empty:
                list_df_flare_data.append(df_flare_data)

        # Guardar resultados del mes concatenando con existentes
        if list_df_full:
            df_full_to_save = pd.concat([existing_full] + list_df_full, ignore_index=True) if not existing_full.empty else pd.concat(list_df_full, ignore_index=True)
            # Asegurarse que la columna 'date' sea la primera
            cols = df_full_to_save.columns.tolist()
            if 'date' in cols:
                cols.insert(0, cols.pop(cols.index('date')))
            df_full_to_save = df_full_to_save[cols]

            df_full_to_save.to_csv(file_full_month, index=False)
            new_downloaded = df_month_to_download['start_time'].dt.date.tolist()
            print(f"💾 df_full_{month}.csv actualizado ({len(df_full_to_save)} filas)")
            print(f"✅ Días descargados este mes: {new_downloaded}")
        elif existing_full.empty:
            pd.DataFrame(columns=['date']).to_csv(file_full_month, index=False)
            print(f"ℹ️ df_full_{month}.csv vacío")

        if list_df_flare_data:
            df_flares_to_save = pd.concat([existing_flares] + list_df_flare_data, ignore_index=True) if not existing_flares.empty else pd.concat(list_df_flare_data, ignore_index=True)
            df_flares_to_save.to_csv(file_flare_month, index=False)
            print(f"💾 df_flare_data_{month}.csv actualizado ({len(df_flares_to_save)} filas)")
        elif existing_flares.empty:
            pd.DataFrame(columns=['StartTime']).to_csv(file_flare_month, index=False)
            print(f"ℹ️ df_flare_data_{month}.csv vacío")

        print(f"🏁 Mes {month} terminado → {month_dir}")



# corre todo

In [7]:
process_goes_by_month_specific_days(df_fechas, subfolder)


📅 Total de días a descargar: 185

🟦 Procesando mes 2000_06 (3 días)
⚠️ Días ya descargados y saltados: [datetime.date(2000, 6, 6), datetime.date(2000, 6, 18)]
1. Descargar datos GOES
Buscando datos de: 2000-06-07 00:00:00


Día 1 de 185: 2000-06-07


Descargando datos de 2000-06-07 00:00:00...


Files Downloaded: 100%|██████████| 3/3 [00:00<00:00,  6.09file/s]
2025-10-24 14:53:29 - sunpy - WARNING: SunpyUserWarning: Unknown units for xrsa
2025-10-24 14:53:29 - sunpy - WARNING: SunpyUserWarning: Unknown units for xrsb


Resolution detected: 3.00 s → resampling at 1 min
Gráfica guardada en 2025-10-24_Download_for_specific_days/Data/Month_2000_06/data_graphs/GOES_2000-06-07_00-00-00.png
Observatorio encontrado: GOES-16
Se encontraron datos  GOES para 2000-06-07 00:00:00 - 2000-06-08 00:00:00. Día saltado.

Número de registros: 1440
Columnas disponibles: ['xrsa', 'xrsb']
2. Restar Background
Plot saved at: 2025-10-24_Download_for_specific_days/Data/Month_2000_06/data_graphs/GOES_diff_linear_2000-06-07_00-00-00.png
Plot saved at: 2025-10-24_Download_for_specific_days/Data/Month_2000_06/data_graphs/GOES_diff_log_2000-06-07_00-00-00.png
Plot saved at: 2025-10-24_Download_for_specific_days/Data/Month_2000_06/data_graphs/GOES_diff_positive_2000-06-07_00-00-00.png
3. USAR FUNCIÓN SUNPY calculate_t_em
Ahora vamos a calcular la T y EM con el modelo de abundancias:coronal


...

se calculó T y EM con el modelo de abundancias:coronal
Ahora vamos a calcular la T y EM con el modelo de abundancias:photospheric


Files Downloaded: 100%|██████████| 1/1 [00:00<00:00,  1.24file/s]
2025-10-24 15:35:15 - root - INFO: Searching for GOES flares between 2025-06-19 00:00:00 and 2025-06-20 00:00:00...


se calculó T y EM con el modelo de abundancias:photospheric
4. CONSTRUIR df_full
5. añadiendo observatorio: GOES: GOES-18
se añadió observatorio: GOES: GOES-18
6. Descargando flares: 2025-06-19 00:00:00 - 2025-06-20 00:00:00


2025-10-24 15:35:17 - root - INFO: Found 8 GOES solar flares between 2025-06-19 00:00:00 and 2025-06-20 00:00:00.


Se descargaron flares para el intervalo 2025-06-19 00:00:00 - 2025-06-20 00:00:00.
7. Se filtraron solo flares de GOES.
💾 df_full_2025_06.csv actualizado (2880 filas)
✅ Días descargados este mes: [datetime.date(2025, 6, 17), datetime.date(2025, 6, 19)]
💾 df_flare_data_2025_06.csv actualizado (16 filas)
🏁 Mes 2025_06 terminado → 2025-10-24_Download_for_specific_days/Data/Month_2025_06


# combinar

In [8]:
combined = combine_blocks(subfolder, f"df_full", f"all_df_full.csv", time_column=None)
combined_flares = combine_blocks(subfolder, f"df_flare_data", f"all_df_flare_data.csv", time_column="StartTime")



⚠️ Hay diferencias en la resolución temporal (no todos los pasos son 1 min).
✅ all_df_full.csv creado con 266400 filas (desde 80 archivos).
⚠️ Hay diferencias en la resolución temporal (no todos los pasos son 1 min).
✅ all_df_flare_data.csv creado con 2022 filas (desde 80 archivos).


In [9]:
# Asegúrate de que 'delta' esté calculado
combined['delta'] = combined['date'].diff()

# Filas donde el salto no es 1 minuto
saltos = combined[combined['delta'] != pd.Timedelta(minutes=1)]

print(saltos[['date', 'delta']])


# %%
print(combined['delta'].value_counts())


# %%
saltos = combined[combined['delta'] != pd.Timedelta(minutes=1)]
# Muestra la fecha de la fila con el salto
print(saltos['date'])


             date            delta
0      2000-06-06              NaT
1440   2000-06-08  1 days 00:01:00
4320   2000-06-18  9 days 00:01:00
5760   2000-07-11 22 days 00:01:00
8640   2000-07-14  1 days 00:01:00
...           ...              ...
259200 2025-03-28 32 days 00:01:00
260640 2025-05-13 45 days 00:01:00
263520 2025-05-25 10 days 00:01:00
264960 2025-06-17 22 days 00:01:00
266400 2025-06-19  1 days 00:01:00

[148 rows x 2 columns]
delta
0 days 00:01:00      266252
1 days 00:01:00          22
2 days 00:01:00          17
3 days 00:01:00           7
6 days 00:01:00           5
                      ...  
128 days 00:01:00         1
35 days 00:01:00          1
12 days 00:01:00          1
49 days 00:01:00          1
32 days 00:01:00          1
Name: count, Length: 68, dtype: int64
0        2000-06-06
1440     2000-06-08
4320     2000-06-18
5760     2000-07-11
8640     2000-07-14
            ...    
259200   2025-03-28
260640   2025-05-13
263520   2025-05-25
264960   2025-06-17
2664